<a href="https://colab.research.google.com/github/Ooojulian/Aquitec/blob/main/notebook/udacity_cs344_hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
# Celda 1: Clonar Repositorio
!git clone https://github.com/depctg/udacity-cs344-colab
%cd udacity-cs344-colab
print("Repositorio clonado y directorio cambiado.")

Cloning into 'udacity-cs344-colab'...
remote: Enumerating objects: 144, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 144 (delta 14), reused 12 (delta 12), pack-reused 120 (from 1)
Receiving objects: 100% (144/144), 3.93 MiB | 35.97 MiB/s, done.
Resolving deltas: 100% (43/43), done.
/content/udacity-cs344-colab/build/udacity-cs344-colab/build/build/udacity-cs344-colab
Repositorio clonado y directorio cambiado.


In [52]:
# Celda 2: Instalar y Cargar Plugin
!pip install git+https://github.com/depctg/nvcc4jupyter.git
# Configurar y cargar el plugin CUDA
%config NVCCPluginV2.static_dir = True
%config NVCCPluginV2.relative_dir = "src/HW4" # Ruta relativa desde /content/udacity-cs344-colab
%load_ext nvcc_plugin
print("Plugin instalado y cargado.")

  Cloning https://github.com/depctg/nvcc4jupyter.git to /tmp/pip-req-build-o3sjkscz
  Running command git clone --filter=blob:none --quiet https://github.com/depctg/nvcc4jupyter.git /tmp/pip-req-build-o3sjkscz
  Resolved https://github.com/depctg/nvcc4jupyter.git to commit 63b811d987446c2a3783845b7b85cda6e562b71a
  Preparing metadata (setup.py) ... done
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin
Plugin instalado y cargado.


In [53]:
# Celda 3: Aplicar TODOS los Parches y Renombrado (Revisado v2)
import os # Necesario para la comprobación de archivo

print("--- 1. Parcheando Arquitecturas ---")
# Usamos punto y coma para múltiples comandos sed en una línea
!sed -i 's/compute_30/compute_75/g; s/sm_30/sm_75/g; s/compute_35/compute_75/g; s/sm_35/sm_75/g' src/HW4/CMakeLists.txt src/CMakeLists.txt
print("Arquitecturas parcheadas.")
# Verification
!grep -rE 'compute_30|sm_30|compute_35|sm_35' src/ || echo "OK: No se encontraron arquitecturas _30 o _35."

print("\n--- 2. Renombrando main.cpp a main.cu (Ignorando error si ya está hecho) ---")
# Intentar renombrar. Si ya es main.cu, mostrará un error 'cannot stat', pero podemos ignorarlo.
!mv src/HW4/main.cpp src/HW4/main.cu
print("Intento de renombrado completado (ignorar error 'cannot stat' si aparece).")

print("\n--- 3. Actualizando referencias a main.cpp en HW4/CMakeLists.txt ---")
# Aplicar el reemplazo 'main.cpp' -> 'main.cu' en todo el archivo HW4/CMakeLists.txt
!sed -i 's/main\.cpp/main.cu/g' src/HW4/CMakeLists.txt
print("Intento de actualizar referencias a main.cpp.")
# Verification - Buscar main.cu ahora
print("Verificando si main.cu aparece en HW4/CMakeLists.txt:")
!grep 'main\.cu' src/HW4/CMakeLists.txt || echo "AVISO: main.cu no encontrado en HW4/CMakeLists.txt después del reemplazo."
# Verificar que main.cpp ya no está
print("Verificando si main.cpp todavía aparece en HW4/CMakeLists.txt:")
!grep 'main\.cpp' src/HW4/CMakeLists.txt || echo "OK: main.cpp no encontrado en HW4/CMakeLists.txt después del reemplazo."


print("\n--- 4. Añadiendo include a HW4.cu (si no existe ya) ---")
# Usar f-string para insertar correctamente las variables de Python en el comando bash
INCLUDE_LINE="#include <thrust/host_vector.h>"
FILE_TO_PATCH="src/HW4/HW4.cu"
# Verificar si el archivo existe antes de intentar parchearlo
if os.path.exists(FILE_TO_PATCH):
    # Comando Bash: Si grep no encuentra (-q) la línea exacta (-F), entonces ejecuta sed para añadirla después de <stdio.h>
    # Usamos f-string para sustituir las variables de Python
    BASH_COMMAND = f"grep -qF '{INCLUDE_LINE}' '{FILE_TO_PATCH}' || sed -i '/<stdio.h>/a {INCLUDE_LINE}' '{FILE_TO_PATCH}'"
    print(f"Ejecutando comando bash para parchear {FILE_TO_PATCH}...")
    # Ejecutar el comando bash
    process_result = subprocess.run(['bash', '-c', BASH_COMMAND], capture_output=True, text=True)
    if process_result.returncode != 0:
        print(f"  AVISO: Comando bash falló con error: {process_result.stderr}")
    else:
        print(f"  Comando bash ejecutado.")

    print("  Verificación/parcheo de HW4.cu completado.")
    # Verification (usando grep directamente con !)
    print(f"  Verificando presencia de '{INCLUDE_LINE}' en {FILE_TO_PATCH}:")
    !grep -F "{INCLUDE_LINE}" "{FILE_TO_PATCH}" || echo "  AVISO: Include '{INCLUDE_LINE}' no encontrado en {FILE_TO_PATCH}"
else:
    print(f"AVISO: El archivo '{FILE_TO_PATCH}' no existe, no se pudo parchear.")

# Añadir import subprocess al principio si no está (aunque no es estrictamente necesario para el !bash -c)
import subprocess

print("\n=== Todos los parches aplicados ===")

--- 1. Parcheando Arquitecturas ---
Arquitecturas parcheadas.
OK: No se encontraron arquitecturas _30 o _35.

--- 2. Renombrando main.cpp a main.cu (Ignorando error si ya está hecho) ---
Intento de renombrado completado (ignorar error 'cannot stat' si aparece).

--- 3. Actualizando referencias a main.cpp en HW4/CMakeLists.txt ---
Intento de actualizar referencias a main.cpp.
Verificando si main.cu aparece en HW4/CMakeLists.txt:
SET (HW4_files main.cu loadSaveImage.cpp reference_calc.cpp compare.cpp)
Verificando si main.cpp todavía aparece en HW4/CMakeLists.txt:
OK: main.cpp no encontrado en HW4/CMakeLists.txt después del reemplazo.

--- 4. Añadiendo include a HW4.cu (si no existe ya) ---
Ejecutando comando bash para parchear src/HW4/HW4.cu...
  Comando bash ejecutado.
  Verificación/parcheo de HW4.cu completado.
  Verificando presencia de '#include <thrust/host_vector.h>' en src/HW4/HW4.cu:
#include <thrust/host_vector.h>

=== Todos los parches aplicados ===


In [54]:
# Celda 4: Configurar CMake

# Crear directorio de build (si no existe) y entrar
!mkdir -p build
%cd build
# Ejecutar CMake
print("\nEjecutando CMake...")
!cmake ../src
# Mensaje para verificar la salida
print("\n=== Verificación de CMake: Busca '-- Build files have been written to: ...' arriba. Si ves errores de CMake, detente. ===")

/content/udacity-cs344-colab/build/udacity-cs344-colab/build/build/udacity-cs344-colab/build

Ejecutando CMake...
CMake Deprecation Warning at CMakeLists.txt:8 (cmake_minimum_required):
  Compatibility with CMake < 3.10 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value.  Or, use the <min>...<max> syntax
  to tell CMake that the project requires at least <min> but has been updated
  to work with policies introduced by <max> or earlier.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Found Op

In [55]:
%%cuda --name student_func.cu

// Udacity HW 4
// Radix Sorting - Implementación en student_func.cu

#include "utils.h"
#include <cuda_runtime.h>
#include <device_launch_parameters.h>
#include <thrust/device_ptr.h>
#include <thrust/scan.h>
#include <thrust/execution_policy.h> // <--- Incluir esto

// --- KERNELS Y FUNCIÓN your_sort(...) ---
// ... (Pega tu implementación completa aquí) ...
// Ejemplo (Asegúrate de tener tu código completo):
__global__ void kernel_histograma(const unsigned int* d_inputVals, unsigned int* d_histograma, const size_t numElems, const int bits_inicio, const int num_bits){/*...*/}
__global__ void kernel_reordenar(const unsigned int* d_inputVals, const unsigned int* d_inputPos, unsigned int* d_outputVals, unsigned int* d_outputPos, const unsigned int* d_offsets, unsigned int* d_output_counts, const size_t numElems, const int bits_inicio, const int num_bits){/*...*/}
void your_sort(unsigned int* const d_inputVals, unsigned int* const d_inputPos, unsigned int* const d_outputVals, unsigned int* const d_outputPos, const size_t numElems){/* TU CÓDIGO COMPLETO AQUÍ */}

'File written in /content/udacity-cs344-colab/src/HW4/student_func.cu'

In [56]:
# Celda 6: Compilar y Ejecutar

# Estamos dentro de /content/udacity-cs344-colab/build
print("Intentando compilar con 'make HW4'...")
!make HW4

print("\nIntentando ejecutar...")
# Ejecutar el programa compilado si make tuvo éxito
!bin/HW4 ../src/HW4/red_eye_effect.gold ../src/HW4/red_eye_effect_template_5.jpg
print("\n====== RESULTADO DE HW4 (busca PASSED o FAILED arriba) =======")

Intentando compilar con 'make HW4'...
make[3]: *** No rule to make target 'HW4/CMakeFiles/HW4.dir/HW4_generated_student_func.cu.o.cmake', needed by 'HW4/CMakeFiles/HW4.dir/HW4_generated_student_func.cu.o'.  Stop.
make[2]: *** [CMakeFiles/Makefile2:316: HW4/CMakeFiles/HW4.dir/all] Error 2
make[1]: *** [CMakeFiles/Makefile2:324: HW4/CMakeFiles/HW4.dir/rule] Error 2
make: *** [Makefile:163: HW4] Error 2

Intentando ejecutar...
/bin/bash: line 1: bin/HW4: No such file or directory

====== RESULTADO DE HW4 (busca PASSED o FAILED arriba) =======


In [57]:
# Celda 7: Visualización

import matplotlib.pyplot as plt
import os

# Asegurarse de estar en el directorio build para encontrar el output
# %cd /content/udacity-cs344-colab/build # Descomentar si es necesario

output_file = "HW4_output.png"
input_file = "../src/HW4/red_eye_effect_template_5.jpg"

print(f"Buscando archivo de salida: {os.path.abspath(output_file)}")
print(f"Buscando archivo de entrada: {os.path.abspath(input_file)}")

if os.path.exists(output_file) and os.path.exists(input_file):
  _,ax = plt.subplots(1,2, dpi=150)
  ax[0].imshow(plt.imread(input_file))
  ax[0].set_title("Original")
  ax[0].grid(False); ax[0].axis('off')
  ax[1].imshow(plt.imread(output_file))
  ax[1].set_title("Salida (Output)")
  ax[1].grid(False); ax[1].axis('off')
  plt.tight_layout(); plt.show()
elif not os.path.exists(input_file):
    print(f"Error: No se encontró el archivo de ENTRADA '{input_file}'. Verifica la ruta.")
elif not os.path.exists(output_file):
    print(f"Error: No se encontró el archivo de SALIDA '{output_file}'.")
    print("Causa más probable: La Celda 6 (make / run) falló. Revisa su salida.")
else:
    print(f"Error: No se encontró el archivo de entrada '{input_file}' o el de salida '{output_file}'.")

Buscando archivo de salida: /content/udacity-cs344-colab/build/udacity-cs344-colab/build/build/udacity-cs344-colab/build/HW4_output.png
Buscando archivo de entrada: /content/udacity-cs344-colab/build/udacity-cs344-colab/build/build/udacity-cs344-colab/src/HW4/red_eye_effect_template_5.jpg
Error: No se encontró el archivo de SALIDA 'HW4_output.png'.
Causa más probable: La Celda 6 (make / run) falló. Revisa su salida.


In [58]:
# Celda 4: Visualización

import matplotlib.pyplot as plt
import os # Para chequear si el archivo existe

output_file = "HW4_output.png"
# Asegúrate que el nombre del archivo de entrada coincide con el usado en Celda 3
input_file = "../src/HW4/red_eye_effect_template_5.jpg"

# Verificar si el archivo de salida existe
if os.path.exists(output_file) and os.path.exists(input_file):
  _,ax = plt.subplots(1,2, dpi=150)

  # Mostrar imagen original
  ax[0].imshow(plt.imread(input_file))
  ax[0].set_title("Original")
  ax[0].grid(False)
  ax[0].axis('off') # Ocultar ejes

  # Mostrar imagen de salida
  ax[1].imshow(plt.imread(output_file))
  ax[1].set_title("Salida (Output)")
  ax[1].grid(False)
  ax[1].axis('off') # Ocultar ejes

  plt.tight_layout() # Ajustar espaciado
  plt.show()
elif not os.path.exists(input_file):
    print(f"Error: No se encontró el archivo de ENTRADA '{input_file}'. Verifica la ruta.")
elif not os.path.exists(output_file):
    print(f"Error: No se encontró el archivo de SALIDA '{output_file}'.")
    print("Esto usualmente significa que la Celda 3 falló (error de compilación o ejecución). Revisa la salida de la Celda 3.")
else:
    print(f"Error: No se encontró el archivo de entrada '{input_file}' o el de salida '{output_file}'.")

Error: No se encontró el archivo de SALIDA 'HW4_output.png'.
Esto usualmente significa que la Celda 3 falló (error de compilación o ejecución). Revisa la salida de la Celda 3.
